In [1]:
import pandas as pd
import duckdb
from pyspainmobility import Mobility, Zones
import seaborn as sns
import matplotlib.pyplot as plt


Leemos primero los datos de los distritos:

In [ ]:
# getting 1 week of mobility data. In this case, we download the data from November 13 to November 19 in 2023 as it was 2 years ago exactly ensuring it goes from Monday to Sunday.
mobility_data_distritos = Mobility(version=2, zones='distritos', start_date='2023-11-13', end_date='2023-11-19')
# and we extract the OD matrices 
mobility_data_distritos.get_od_data()


In [3]:
df_distritos = pd.read_parquet('C:/Users/carlo/data/Viajes_distritos_2023-11-13_2023-11-19_v2.parquet')
print(df_distritos)


                date  hour id_origin id_destination  n_trips  \
0         2023-11-13     0     01001          01001   13.064   
1         2023-11-13     0     01001       01009_AM   16.660   
2         2023-11-13     0     01001       01058_AM    6.720   
3         2023-11-13     0     01001        0105902   10.807   
4         2023-11-13     0     01001        0105903   15.979   
...              ...   ...       ...            ...      ...   
22337972  2023-11-19    23   externo          43901    5.814   
22337973  2023-11-19    23   externo        4801308    1.000   
22337974  2023-11-19    23   externo        4802002    3.460   
22337975  2023-11-19    23   externo       49261_AM    2.488   
22337976  2023-11-19    23   externo        5029707    1.000   

          trips_total_length_km  
0                        52.925  
1                        79.645  
2                        85.175  
3                       153.921  
4                       178.759  
...                        

Luego de los municipios:

In [ ]:
# getting 1 week of mobility data. In this case, we download the data from November 13 to November 19 in 2023 as it was 2 years ago exactly ensuring it goes from Monday to Sunday.
mobility_data_municipios = Mobility(version=2, zones='municipios', start_date='2023-11-13', end_date='2023-11-19')
# and we extract the OD matrices 
mobility_data_municipios.get_od_data()

In [5]:
df_municipios = pd.read_parquet('C:/Users/carlo/data/Viajes_municipios_2023-11-13_2023-11-19_v2.parquet')
print(df_municipios)

                date  hour id_origin id_destination  n_trips  \
0         2023-11-13     0     01001          01001   13.064   
1         2023-11-13     0     01001       01009_AM   16.660   
2         2023-11-13     0     01001       01058_AM    6.720   
3         2023-11-13     0     01001          01059   38.340   
4         2023-11-13     0     01002          01002  264.818   
...              ...   ...       ...            ...      ...   
11408718  2023-11-19    23   externo          43901    5.814   
11408719  2023-11-19    23   externo          48013    1.000   
11408720  2023-11-19    23   externo          48020    3.460   
11408721  2023-11-19    23   externo       49261_AM    2.488   
11408722  2023-11-19    23   externo          50297    1.000   

          trips_total_length_km  
0                        52.925  
1                        79.645  
2                        85.175  
3                       463.621  
4                       972.675  
...                        

Finalmente de los GAUs:

In [ ]:
# YA EJECUTADO

# getting 1 week of mobility data. In this case, we download the data from November 13 to November 19 in 2023 as it was 2 years ago exactly ensuring it goes from Monday to Sunday.
mobility_data_gaus = Mobility(version=2, zones='gaus', start_date='2023-11-13', end_date='2023-11-19')
# and we extract the OD matrices 
mobility_data_gaus.get_od_data()

In [7]:
df_gaus = pd.read_parquet('C:/Users/carlo/data/Viajes_GAU_2023-11-13_2023-11-19_v2.parquet')
print(df_gaus)

               date  hour id_origin                          id_destination  \
0        2023-11-13     0     01001                                   01001   
1        2023-11-13     0     01001                                01009_AM   
2        2023-11-13     0     01001                                01058_AM   
3        2023-11-13     0     01001                     GAU Vitoria/Gasteiz   
4        2023-11-13     0     01002                                   01002   
...             ...   ...       ...                                     ...   
6464589  2023-11-19    23   externo                   GAU Palma de Mallorca   
6464590  2023-11-19    23   externo               GAU Sant Feliú de Guixols   
6464591  2023-11-19    23   externo  GAU Santa Cruz de Tenerife - La Laguna   
6464592  2023-11-19    23   externo                   GAU Vigo - Pontevedra   
6464593  2023-11-19    23   externo                            GAU Zaragoza   

         n_trips  trips_total_length_km  
0        

Ahora unimos las 3 zonas en un mismo dataframe:

In [8]:
df_distritos['zone_level'] = 'distrito'
df_municipios['zone_level'] = 'municipio'
df_gaus['zone_level'] = 'GAU'
df_distritos['source'] = 'MITMA'
df_municipios['source'] = 'MITMA'
df_gaus['source'] = 'MITMA'

od = pd.concat([df_distritos, df_municipios, df_gaus])
print(od.head())

         date  hour id_origin id_destination  n_trips  trips_total_length_km  \
0  2023-11-13     0     01001          01001   13.064                 52.925   
1  2023-11-13     0     01001       01009_AM   16.660                 79.645   
2  2023-11-13     0     01001       01058_AM    6.720                 85.175   
3  2023-11-13     0     01001        0105902   10.807                153.921   
4  2023-11-13     0     01001        0105903   15.979                178.759   

  zone_level source  
0   distrito  MITMA  
1   distrito  MITMA  
2   distrito  MITMA  
3   distrito  MITMA  
4   distrito  MITMA  


Ahora leemos los documentos del INE:

In [9]:
con = duckdb.connect('mobility_populatio.db')
df = pd.read_excel("poblaciones.xlsx", header=None, names=["code_name","population"])
df.to_csv("poblaciones.csv", index=False)
df = pd.read_excel("rentas.xlsx", header=None, names=["code_name","rent"])
df.to_csv("rentas.csv", index=False)

In [10]:
import pandas as pd

# ==========================================
# 1. GENERAR CSV DE POBLACIÓN
# ==========================================
df_pop = pd.read_excel(
    "C:/Users/carlo/Downloads/poblacion_TODO.xlsx",
    header=None,       
    skiprows=2,        # Saltamos los encabezados verdes
    usecols=[0, 2],    # Columna A (Nombre) y C (Dato Población)
    names=["code_name", "population"]
)

# Separar "01001 Alegría" en dos columnas
df_pop[['code', 'name']] = df_pop['code_name'].str.split(' ', n=1, expand=True)

# --- LIMPIEZA ROBUSTA (Anti-errores) ---
# 1. Forzar que 'population' sea numérico. Los espacios " " se vuelven NaN.
df_pop['population'] = pd.to_numeric(df_pop['population'], errors='coerce')
# 2. Borrar filas donde la población sea NaN (elimina basura del final del Excel)
df_pop = df_pop.dropna(subset=['population'])
# 3. Convertir a entero (opcional, queda mejor)
df_pop['population'] = df_pop['population'].astype(int)

# Seleccionar columnas finales en orden limpio
df_pop = df_pop[['code', 'name', 'population']]

# Guardar
df_pop.to_csv("poblaciones.csv", index=False)
print(f"Generado 'poblaciones.csv' con {len(df_pop)} filas.")


# ==========================================
# 2. GENERAR CSV DE RENTA
# ==========================================
df_renta = pd.read_excel(
    "C:/Users/carlo/Downloads/renta_TODO.xlsx",
    header=None,
    skiprows=2,
    usecols=[0, 1],    # Columna A (Nombre) y B (Dato Renta)
    names=["code_name", "rent"]
)

# Separar código y nombre
df_renta[['code', 'name']] = df_renta['code_name'].str.split(' ', n=1, expand=True)

# --- LIMPIEZA ROBUSTA ---
df_renta['rent'] = pd.to_numeric(df_renta['rent'], errors='coerce')
df_renta = df_renta.dropna(subset=['rent'])

# Seleccionar columnas finales
df_renta = df_renta[['code', 'name', 'rent']]

# Guardar
df_renta.to_csv("rentas.csv", index=False)
print(f"Generado 'rentas.csv' con {len(df_renta)} filas.")

Generado 'poblaciones.csv' con 55069 filas.
Generado 'rentas.csv' con 54840 filas.


In [11]:
df_pop.head()

,code,name,population
0,01001,Alegría-Dulantzi,2908
1,0100101,Alegría-Dulantzi distrito 01,2908
2,0100101001,Alegría-Dulantzi sección 01001,1353
3,0100101002,Alegría-Dulantzi sección 01002,1555
4,01002,Amurrio,10278


In [12]:
df_renta.head()

,code,name,rent
0,01001,Alegría-Dulantzi,16429.0
1,0100101,Alegría-Dulantzi distrito 01,16429.0
2,0100101001,Alegría-Dulantzi sección 01001,17069.0
3,0100101002,Alegría-Dulantzi sección 01002,15869.0
4,01002,Amurrio,17486.0


In [13]:
od.head()

,date,hour,id_origin,id_destination,n_trips,trips_total_length_km,zone_level,source
0,2023-11-13,0,01001,01001,13.064,52.925,distrito,MITMA
1,2023-11-13,0,01001,01009_AM,16.660,79.645,distrito,MITMA
2,2023-11-13,0,01001,01058_AM,6.720,85.175,distrito,MITMA
3,2023-11-13,0,01001,0105902,10.807,153.921,distrito,MITMA
4,2023-11-13,0,01001,0105903,15.979,178.759,distrito,MITMA


In [14]:
con.execute("""
CREATE OR REPLACE TABLE viajes_bronze (
    date DATE,
    hour INTEGER,
    id_origin VARCHAR,
    id_destination VARCHAR,
    n_trips DOUBLE,
    trips_total_length_km DOUBLE,
    PRIMARY KEY (date, hour, id_origin, id_destination)
    )
""")


In [15]:
con.execute("""
INSERT INTO viajes_bronze
SELECT * FROM 'C:/Users/carlo/data/Viajes_GAU_2023-11-13_2023-11-19_v2.parquet';
""")


In [16]:
con.execute("SELECT * FROM viajes_bronze LIMIT 5").df()

,date,hour,id_origin,id_destination,n_trips,trips_total_length_km
0,2023-11-13,0,01001,01001,13.064,52.925
1,2023-11-13,0,01001,01009_AM,16.660,79.645
2,2023-11-13,0,01001,01058_AM,6.720,85.175
3,2023-11-13,0,01001,GAU Vitoria/Gasteiz,38.340,463.621
4,2023-11-13,0,01002,01002,264.818,972.675


In [17]:
# Tabla de Población
con.execute("""
CREATE OR REPLACE TABLE poblacion_bronze (
    code VARCHAR,
    name VARCHAR,
    population INTEGER,
    PRIMARY KEY (code)
);
""")


In [18]:
con.execute("COPY poblacion_bronze FROM 'poblaciones.csv' (HEADER, DELIMITER ',', NULL ' ')")

print(con.execute("SELECT * FROM poblacion_bronze LIMIT 5").df())

         code                            name  population
0       01001                Alegría-Dulantzi        2908
1     0100101    Alegría-Dulantzi distrito 01        2908
2  0100101001  Alegría-Dulantzi sección 01001        1353
3  0100101002  Alegría-Dulantzi sección 01002        1555
4       01002                         Amurrio       10278


In [19]:

# Tabla de Renta
con.execute("""
CREATE OR REPLACE TABLE renta_bronze (
    code VARCHAR,
    name VARCHAR,
    rent DOUBLE,
    PRIMARY KEY (code)
);
""")

In [20]:
con.execute("COPY renta_bronze FROM 'rentas.csv' (HEADER, DELIMITER ',', NULL ' ')")

print(con.execute("SELECT * FROM renta_bronze LIMIT 5").df())

         code                            name     rent
0       01001                Alegría-Dulantzi  16429.0
1     0100101    Alegría-Dulantzi distrito 01  16429.0
2  0100101001  Alegría-Dulantzi sección 01001  17069.0
3  0100101002  Alegría-Dulantzi sección 01002  15869.0
4       01002                         Amurrio  17486.0


Ya están las tablas de viajes_bronze, pobacion_bronze y renta_bronze cargadas

AQUÍ ACABA LA FASE BRONCE: HEMOS METIDO TODOS LOS DATOS CRUDOS EN 3 TABLAS DISTINTAS 

Hacemos algunas consultas sencillas para ver que funcione:

In [21]:
con.execute('''
SELECT (*) FROM poblacion_bronze WHERE code = '01001'
''').df()

,code,name,population
0,01001,Alegría-Dulantzi,2908


In [22]:
con.execute('''
SELECT (*) FROM renta_bronze LIMIT 5
''').df()

,code,name,rent
0,01001,Alegría-Dulantzi,16429.0
1,0100101,Alegría-Dulantzi distrito 01,16429.0
2,0100101001,Alegría-Dulantzi sección 01001,17069.0
3,0100101002,Alegría-Dulantzi sección 01002,15869.0
4,01002,Amurrio,17486.0


AHORA PASAMOS A LA FASE SILVER: DATOS LIMPIOS

Primero vamos a crear la tabla relación códigos MITMA e INE

In [23]:
con.execute('''
CREATE OR REPLACE TABLE rel_muni (
    code_ine VARCHAR,
    code_mitma VARCHAR,
    PRIMARY KEY (code_ine, code_mitma)
);
''')

con.execute('''
INSERT INTO rel_muni (code_ine, code_mitma)
SELECT DISTINCT
    municipio,
    municipio_mitma
FROM read_csv_auto("C:/Users/carlo/Downloads/relaciones_municipio_mitma.csv");
''')

con.execute('''
SELECT (*) FROM rel_muni LIMIT 10
''').df()

,code_ine,code_mitma
0,01041,01031_AM
1,02010,02010_AM
2,02014,02008_AM
3,02051,02033_AM
4,02075,02045_AM
5,03001,03028_AM
6,03016,03028_AM
7,03025,03058_AM
8,03036,03028_AM
9,03056,03056


Vamos a crear dos tablas: silver_zone_metrics y silver_integrated_od

In [24]:
con.execute('''
CREATE OR REPLACE TABLE silver_zone_metrics AS
WITH 
-- 1. Limpieza y Deduplicación de Población
clean_pop AS (
    SELECT 
        code, 
        name, 
        population,
        ROW_NUMBER() OVER (PARTITION BY code ORDER BY population DESC) as rn
    FROM poblacion_bronze
    WHERE code IS NOT NULL
),
-- 2. Limpieza y Deduplicación de Renta
clean_rent AS (
    SELECT 
        code, 
        rent,
        ROW_NUMBER() OVER (PARTITION BY code ORDER BY rent DESC) as rn
    FROM renta_bronze
    WHERE code IS NOT NULL
)
-- 3. Integración (Join)
SELECT 
    p.code AS zone_id,
    p.name AS zone_name,
    COALESCE(p.population, 0) AS population, -- Null handling: si es null pone 0
    COALESCE(r.rent, 0) AS avg_rent          -- Null handling
FROM clean_pop p
LEFT JOIN clean_rent r 
    ON p.code = r.code AND r.rn = 1
WHERE p.rn = 1;
''')

print("✅ Tabla 'silver_zone_metrics' creada.")
print(con.execute("SELECT * FROM silver_zone_metrics LIMIT 5").df())

✅ Tabla 'silver_zone_metrics' creada.
      zone_id                    zone_name  population  avg_rent
0     0100201          Amurrio distrito 01       10278   17486.0
1     0101601          Bernedo distrito 01         544   18859.0
2     0101701  Campezo/Kanpezu distrito 01        1088   16466.0
3     0102101  Elburgo/Burgelu distrito 01         641   20447.0
4  0103601004  Laudio/Llodio sección 01004        1263   18391.0


In [25]:
# 1. Visualizamos qué vamos a intentar cruzar (Diagnóstico)
print("Intentando casar nombres de GAU con nombres de Municipios...")

# 2. Inserción Automática (La Magia)
# Buscamos coincidencias por NOMBRE y guardamos los códigos en rel_muni
con.execute('''
INSERT INTO rel_muni (code_mitma, code_ine)
SELECT DISTINCT 
    v.id_origin AS code_mitma,
    p.zone_id AS code_ine
FROM viajes_bronze v
JOIN silver_zone_metrics p 
    -- Aquí está el truco: Limpiamos la basura del string para comparar nombres
    ON TRIM(REPLACE(REPLACE(v.id_origin, 'Zona GAU ', ''), 'GAU ', '')) = p.zone_name
WHERE 
    (v.id_origin LIKE '%GAU%' OR v.id_origin LIKE '%Zona%')
    -- IMPORTANTE: Solo insertamos si ese código NO existe ya en la tabla (evitar duplicados)
    AND v.id_origin NOT IN (SELECT code_mitma FROM rel_muni);
''')

# 3. Verificamos cuántos ha encontrado
nuevos_gau = con.execute("""
    SELECT count(*) FROM rel_muni WHERE code_mitma LIKE '%GAU%'
""").fetchone()[0]

print(f"✅ Se han detectado y mapeado automáticamente {nuevos_gau} zonas GAU.")
print("Ejemplos de lo que ha encontrado:")
print(con.execute("SELECT * FROM rel_muni WHERE code_mitma LIKE '%Guadalajara%' OR code_mitma LIKE '%Madrid%' LIMIT 5").df())

Intentando casar nombres de GAU con nombres de Municipios...
✅ Se han detectado y mapeado automáticamente 58 zonas GAU.
Ejemplos de lo que ha encontrado:
  code_ine       code_mitma
0    28079       GAU Madrid
1    19130  GAU Guadalajara


In [26]:
con.execute('''
CREATE OR REPLACE TABLE silver_integrated_od AS
SELECT 
    v.date, v.hour, v.n_trips, v.trips_total_length_km,
    dayname(v.date) AS day_of_week,
    CASE WHEN dayofweek(v.date) IN (0, 6) THEN TRUE ELSE FALSE END AS is_weekend,

    -- ORIGEN
    v.id_origin AS origin_code_mitma,
    COALESCE(o.zone_name, 'Zona ' || v.id_origin) AS origin_name,
    COALESCE(o.population, 0) AS origin_population, 
    COALESCE(o.avg_rent, 0) AS origin_rent,

    -- DESTINO
    v.id_destination AS destination_code_mitma,
    COALESCE(d.zone_name, 'Zona ' || v.id_destination) AS destination_name,
    COALESCE(d.population, 0) AS destination_population,
    COALESCE(d.avg_rent, 0) AS destination_rent

FROM viajes_bronze v

-- JOIN 1: Traductor (Ahora incluye los GAUs automáticos)
LEFT JOIN rel_muni rel_o ON v.id_origin = rel_o.code_mitma
LEFT JOIN rel_muni rel_d ON v.id_destination = rel_d.code_mitma

-- JOIN 2: Datos
LEFT JOIN silver_zone_metrics o 
    ON COALESCE(rel_o.code_ine, v.id_origin) = o.zone_id
LEFT JOIN silver_zone_metrics d 
    ON COALESCE(rel_d.code_ine, v.id_destination) = d.zone_id

WHERE v.n_trips >= 0;
''')

con.execute("""
    SELECT *
    FROM silver_integrated_od
    LIMIT 10         
""").fetchdf()

,date,hour,n_trips,trips_total_length_km,day_of_week,is_weekend,origin_code_mitma,origin_name,origin_population,origin_rent,destination_code_mitma,destination_name,destination_population,destination_rent
0,2023-11-13,15,5.201,2492.162,Monday,False,GAU Valladolid,Valladolid,296647,16356.0,03092_AM,Alfafara,412,13737.0
1,2023-11-13,15,10.316,654.376,Monday,False,GAU Valladolid,Valladolid,296647,16356.0,05016,Arévalo,7708,13421.0
2,2023-11-13,15,2.060,129.748,Monday,False,GAU Valladolid,Valladolid,296647,16356.0,05114_AM,Fontiveros,727,14233.0
3,2023-11-13,15,7.500,491.843,Monday,False,GAU Valladolid,Valladolid,296647,16356.0,05242_AM,Tiñosillos,759,13089.0
4,2023-11-13,15,5.436,3783.100,Monday,False,GAU Valladolid,Valladolid,296647,16356.0,07055,Santa Margalida,13450,14707.0
5,2023-11-13,15,17.880,1525.062,Monday,False,GAU Valladolid,Valladolid,296647,16356.0,09018,Aranda de Duero,33360,14702.0
6,2023-11-13,15,3.417,310.204,Monday,False,GAU Valladolid,Valladolid,296647,16356.0,09141_AM,Hontangas,99,16232.0
7,2023-11-13,15,3.596,292.005,Monday,False,GAU Valladolid,Valladolid,296647,16356.0,09211_AM,Villamayor de Treviño,56,17307.0
8,2023-11-13,15,4.091,758.171,Monday,False,GAU Valladolid,Valladolid,296647,16356.0,09219,Miranda de Ebro,35745,15262.0
9,2023-11-13,15,7.908,435.264,Monday,False,GAU Valladolid,Valladolid,296647,16356.0,09321_AM,Pedrosa de Duero,461,14396.0


In [27]:
con.execute("""
SELECT DISTINCT 
    origin_code_mitma, 
    origin_name, 
    origin_population 
FROM silver_integrated_od 
WHERE origin_code_mitma = '28079' OR origin_name LIKE '%Madrid%'
""").df()

,origin_code_mitma,origin_name,origin_population
0,GAU Madrid,Madrid,3362335
1,45087,Madridejos,10037


todas los  municipios con poblacion y renta

In [28]:
# Consulta simple a la tabla maestra de municipios
con.execute("""
SELECT 
    zone_id, 
    zone_name, 
    population, 
    avg_rent 
FROM silver_zone_metrics
ORDER BY population DESC -- Ordenado por habitantes para ver los grandes primero
LIMIT 20
""").fetchdf()

,zone_id,zone_name,population,avg_rent
0,28079,Madrid,3362335,19632.0
1,08019,Barcelona,1613579,19527.0
2,46250,València,813957,16121.0
3,50297,Zaragoza,682727,16272.0
4,41091,Sevilla,680373,14979.0
5,29067,Málaga,584203,13847.0
6,30030,Murcia,467316,13906.0
7,07040,Palma,425745,16181.0
8,35016,"Palmas de Gran Canaria, Las",376986,14968.0
9,03014,Alacant/Alicante,354576,13681.0


Poblaciones con renta por encima de la media

In [29]:
con.execute("""
SELECT 
    zone_id, 
    zone_name, 
    population, 
    avg_rent 
FROM silver_zone_metrics
WHERE avg_rent > (
    -- Subconsulta: Calcula la media de todos los municipios (ignorando los que tienen 0)
    SELECT AVG(avg_rent) 
    FROM silver_zone_metrics 
    WHERE avg_rent > 0
)
ORDER BY avg_rent DESC -- Mostramos los más ricos primero
LIMIT 20;
""").df()

,zone_id,zone_name,population,avg_rent
0,2800601036,Alcobendas sección 01036,2034,36918.0
1,4625002036,València sección 02036,786,36918.0
2,2811501002,Pozuelo de Alarcón sección 01002,2098,36918.0
3,2807908175,Madrid sección 08175,1220,36918.0
4,0801904056,Barcelona sección 04056,1644,36918.0
5,2807905072,Madrid sección 05072,1648,36918.0
6,0801905049,Barcelona sección 05049,1701,36918.0
7,2807904011,Madrid sección 04011,1400,36918.0
8,4802006008,Bilbao sección 06008,728,36918.0
9,4802006012,Bilbao sección 06012,859,36918.0


par origen-destino con más viajes

In [30]:
con.execute("""
SELECT 
    origin_name,
    destination_name,
    SUM(n_trips) AS total_trips,
    -- Calculamos la media: Total Km / Total Viajes
    SUM(trips_total_length_km) / SUM(n_trips) AS avg_distance_km
FROM silver_integrated_od
GROUP BY origin_name, destination_name
ORDER BY total_trips DESC
LIMIT 5
""").fetchdf()

,origin_name,destination_name,total_trips,avg_distance_km
0,Madrid,Madrid,1.266418e+08,5.030676
1,Barcelona,Barcelona,1.064097e+08,4.530546
2,Zona GAU Valencia,Zona GAU Valencia,3.072412e+07,3.202787
3,Sevilla,Sevilla,2.632298e+07,3.545643
4,Málaga,Málaga,2.070982e+07,3.708123


ruta Origen-Destino con menor flujo de viajes registrado

In [31]:
con.execute('''
SELECT
    origin_name,
    destination_name,
    SUM(n_trips) AS total_trips
FROM silver_integrated_od
-- Agrupamos por nombres para sumar todos los viajes entre esos dos municipios
GROUP BY origin_name, destination_name
-- Importante: Filtramos que la suma sea > 0 para encontrar la ruta "existente" con menos viajes
HAVING total_trips > 0
ORDER BY total_trips ASC
LIMIT 10;
''').df()


,origin_name,destination_name,total_trips
0,Cabreros del Río,Valdevacas y Guijar,1.0
1,Trasobares,Zona FRE11,1.0
2,Zona FRE11,Guadalmez,1.0
3,Zona FRD21,"Picazo, El",1.0
4,Narrillos del Rebollar,Cardona,1.0
5,Monterde de Albarracín,Pinilla de los Barruecos,1.0
6,Tauste,Barracas,1.0
7,Villanueva de la Sierra,Zona PT16E,1.0
8,Herreros de Suso,Sant Mateu de Bages,1.0
9,Gisclareny,Vilallonga del Camp,1.0


conexión principal entre grandes urbes (>300k hab.)

In [32]:
con.execute('''
SELECT 
    v.origin_name,
    -- 1. Población del Origen (Alias p_orig)
    p_orig.population AS poblacion_origen,
    
    v.destination_name,
    -- 2. Población del Destino (Alias p_dest)
    p_dest.population AS poblacion_destino,
    
    SUM(v.n_trips) as total_trips

FROM silver_integrated_od v

-- Primer Join: Buscamos datos del ORIGEN
LEFT JOIN silver_zone_metrics p_orig 
    ON v.origin_name = p_orig.zone_name

-- Segundo Join: Buscamos datos del DESTINO
LEFT JOIN silver_zone_metrics p_dest 
    ON v.destination_name = p_dest.zone_name

WHERE v.origin_name = 'Madrid' -- O el filtro que quieras
GROUP BY 1, 2, 3, 4
ORDER BY total_trips DESC
LIMIT 5
''').df()

,origin_name,poblacion_origen,destination_name,poblacion_destino,total_trips
0,Madrid,3362335,Madrid,3362335,1.266418e+08
1,Madrid,3362335,Guadalajara,89520,3.442668e+05
2,Madrid,3362335,Alpedrete,15380,1.860602e+05
3,Madrid,3362335,Valdemorillo,13972,1.440025e+05
4,Madrid,3362335,Illescas,31687,1.422174e+05


Top Rutas entre Grandes Ciudades

In [33]:
con.execute('''
SELECT
    -- Datos del Origen
    p_orig.zone_name AS origin_name,
    p_dest.zone_name AS dest_name,
    
    -- Poblaciones Reales (Traídas de la tabla maestra)
    p_orig.population AS origin_pop,
    p_dest.population AS dest_pop,
    
    -- Volumen de viajes
    SUM(v.n_trips) AS total_trips

FROM silver_integrated_od v

-- 1. Cruzamos con Población Origen (INNER JOIN para filtrar solo las que existen)
JOIN silver_zone_metrics p_orig 
    ON v.origin_name = p_orig.zone_name

-- 2. Cruzamos con Población Destino
JOIN silver_zone_metrics p_dest 
    ON v.destination_name = p_dest.zone_name

WHERE 
    p_orig.population > 100000      -- Origen Grande
    AND p_dest.population > 100000  -- Destino Grande
    AND p_orig.zone_name != p_dest.zone_name -- Excluir viajes internos (dentro de la misma ciudad)

GROUP BY 1, 2, 3, 4
ORDER BY total_trips DESC
LIMIT 10;
''').df()

,origin_name,dest_name,origin_pop,dest_pop,total_trips
0,Roquetas de Mar,Almería,106855,199568,170850.962
1,Almería,Roquetas de Mar,199568,106855,169935.852
2,Cartagena,Murcia,216385,467316,149872.191
3,Murcia,Cartagena,467316,216385,149714.872
4,Barcelona,Madrid,1613579,3362335,79614.910
5,Girona,Barcelona,103458,1613579,77447.073
6,Barcelona,Girona,1613579,103458,77277.502
7,Madrid,Barcelona,3362335,1613579,76486.750
8,Donostia/San Sebastián,Bilbao,185036,343552,44675.291
9,Bilbao,Donostia/San Sebastián,343552,185036,42885.012


AHORA PASAMOS A LA FASE DE ORO: RESPONDEREMOS A PREGUNTAS DE NEGOCIO

Primero creamos la tabla de la capa de oro:

In [34]:
con.execute('''
CREATE OR REPLACE TABLE gold_analisis_infraestructura AS
WITH calculos_basicos AS (
    SELECT 
        v.origin_name,
        v.n_trips AS viajes_reales,
        
        -- FÓRMULA DE GRAVEDAD (Simplificada)
        -- Cuanta más gente y más cerca, más alto es este número ("Puntos de Gravedad")
        (CAST(p1.population AS BIGINT) * CAST(p2.population AS BIGINT)) / 
        NULLIF(POWER(v.trips_total_length_km / NULLIF(v.n_trips, 0), 2), 0) AS demanda_teorica_puntos

    FROM silver_integrated_od v
    -- Unimos para sacar la población (usando INNER JOIN para ignorar errores)
    JOIN silver_zone_metrics p1 ON v.origin_name = p1.zone_name
    JOIN silver_zone_metrics p2 ON v.destination_name = p2.zone_name
    
    WHERE v.origin_name != v.destination_name -- Ignoramos viajes internos
      AND v.n_trips > 0
)

SELECT 
    origin_name AS municipio,
    
    -- 1. Realidad
    SUM(viajes_reales) AS total_viajes_reales,
    
    -- 2. Teoría (Suma de puntos)
    CAST(SUM(demanda_teorica_puntos) AS BIGINT) AS potencial_teorico,
    
    -- 3. EL ÍNDICE (Ranking)
    -- Dividimos Realidad / Teoría. 
    -- Cuanto más BAJO sea este número, PEOR es la infraestructura.
    ROUND((SUM(viajes_reales) / NULLIF(SUM(demanda_teorica_puntos), 0)) * 1000000, 2) AS indice_cobertura

FROM calculos_basicos
GROUP BY origin_name
HAVING total_viajes_reales > 500 -- Filtramos pueblos muy pequeños
ORDER BY indice_cobertura ASC; -- LOS PEORES ARRIBA
''')

con.execute("""
    SELECT *  
    FROM gold_analisis_infraestructura
    LIMIT 10 
""").fetchdf()

,municipio,total_viajes_reales,potencial_teorico,indice_cobertura
0,Centelles,46827.976,137277692152,0.34
1,Alpedrete,274704.737,196399413188,1.40
2,Cubas de la Sagra,91212.722,63604444060,1.43
3,Moralzarzal,168323.820,115268778483,1.46
4,Torrejón de Velasco,71999.061,42672138873,1.69
5,Hoyo de Manzanares,90847.724,53492781326,1.70
6,Valdeolmos-Alalpardo,63546.572,29511936144,2.15
7,"Montesinos, Los",17847.948,7900291235,2.26
8,Casarrubuelos,64588.698,27270345405,2.37
9,Quijorna,38230.622,15656642550,2.44


¿Qué zonas con una población mayor a 20k tienen peor servicio de infraestructura?

In [35]:
print("--- Grandes Ciudades (>20k) con peor infraestructura de transporte ---")

con.execute("""
SELECT 
    g.municipio,
    p.population AS poblacion, -- Mostramos la población para verificar
    g.total_viajes_reales,
    g.indice_cobertura
FROM gold_analisis_infraestructura g
-- Cruzamos con la tabla Silver para recuperar el dato de habitantes y filtrar
JOIN silver_zone_metrics p 
    ON g.municipio = p.zone_name
WHERE p.population >20000 -- <--- AQUÍ ESTÁ TU FILTRO
ORDER BY g.indice_cobertura ASC -- Los peores arriba
LIMIT 10
""").df()

--- Grandes Ciudades (>20k) con peor infraestructura de transporte ---


,municipio,poblacion,total_viajes_reales,indice_cobertura
0,Seseña,29417,2.378918e+05,4.09
1,Madrid,3362335,1.104106e+07,7.06
2,Aranjuez,61503,3.292163e+05,7.18
3,Algemesí,27834,1.782726e+05,7.23
4,Calafell,31390,3.363710e+05,7.26
5,Ceuta,79953,6.829591e+03,7.89
6,Pilar de la Horadada,23579,1.904626e+05,8.07
7,Illescas,31687,4.578551e+05,8.13
8,Orihuela,82981,9.182096e+05,8.16
9,Utrera,52013,3.157329e+05,8.22


Vamos a hacer un análisis de movilidad a nivel provincial. Para ello, vamos a agregar los municipios, distritos y GAUs por provincias.

In [36]:
con.execute("DROP TABLE IF EXISTS dim_provincias")
con.execute("CREATE TABLE dim_provincias (code VARCHAR, name VARCHAR)")

# Insertamos los datos (He formateado tu lista para SQL)
con.execute("""
INSERT INTO dim_provincias VALUES 
('01','Álava'), ('02','Albacete'), ('03','Alicante'), ('04','Almería'), ('05','Ávila'),
('06','Badajoz'), ('07','Baleares'), ('08','Barcelona'), ('09','Burgos'), ('10','Cáceres'),
('11','Cádiz'), ('12','Castellón'), ('13','Ciudad Real'), ('14','Córdoba'), ('15','La Coruña'),
('16','Cuenca'), ('17','Gerona'), ('18','Granada'), ('19','Guadalajara'), ('20','Guipúzcoa'),
('21','Huelva'), ('22','Huesca'), ('23','Jaén'), ('24','León'), ('25','Lérida'),
('26','La Rioja'), ('27','Lugo'), ('28','Madrid'), ('29','Málaga'), ('30','Murcia'),
('31','Navarra'), ('32','Orense'), ('33','Asturias'), ('34','Palencia'), ('35','Las Palmas'),
('36','Pontevedra'), ('37','Salamanca'), ('38','Santa Cruz de Tenerife'), ('39','Cantabria'),
('40','Segovia'), ('41','Sevilla'), ('42','Soria'), ('43','Tarragona'), ('44','Teruel'),
('45','Toledo'), ('46','Valencia'), ('47','Valladolid'), ('48','Vizcaya'), ('49','Zamora'),
('50','Zaragoza'), ('51','Ceuta'), ('52','Melilla');
""")
print("✅ Tabla maestra de provincias creada.")

✅ Tabla maestra de provincias creada.


In [37]:
con.execute('''
CREATE OR REPLACE TABLE gold_flujos_provinciales AS
SELECT 
    -- 1. Identificamos Provincia Origen (Primeros 2 dígitos del código MITMA)
    po.name AS provincia_origen,
    
    -- 2. Identificamos Provincia Destino
    pd.name AS provincia_destino,
    
    -- 3. Métricas Agregadas
    SUM(v.n_trips) AS viajes_totales,
    ROUND(SUM(v.trips_total_length_km) / NULLIF(SUM(v.n_trips), 0), 2) AS distancia_media_km

FROM silver_integrated_od v
-- Join para nombre de Provincia Origen
LEFT JOIN dim_provincias po 
    ON SUBSTR(v.origin_code_mitma, 1, 2) = po.code
-- Join para nombre de Provincia Destino
LEFT JOIN dim_provincias pd 
    ON SUBSTR(v.destination_code_mitma, 1, 2) = pd.code

WHERE 
    po.name IS NOT NULL 
    AND pd.name IS NOT NULL
    AND po.name != pd.name -- Excluir viajes internos (dentro de la misma provincia)

GROUP BY 1, 2
ORDER BY viajes_totales DESC;
''')

print("🏆 Tabla GOLD 'gold_flujos_provinciales' creada.")

🏆 Tabla GOLD 'gold_flujos_provinciales' creada.


In [ ]:
print("--- Pares de Provincias con MENOS interacción ---")
con.execute("""
SELECT 
    provincia_origen,
    provincia_destino,
    viajes_totales,
    distancia_media_km
FROM gold_flujos_provinciales
ORDER BY viajes_totales ASC
LIMIT 20
""").df()

--- Pares de Provincias con MENOS interacción ---


,provincia_origen,provincia_destino,viajes_totales,distancia_media_km
0,Álava,Málaga,2.148,650.27
1,Córdoba,Álava,2.371,543.15
2,Ávila,Almería,2.376,462.47
3,Lugo,Gerona,3.041,870.11
4,Almería,Cantabria,3.265,697.55
5,La Coruña,Huesca,3.329,713.75
6,Pontevedra,Teruel,3.333,655.66
7,Lérida,La Coruña,3.466,803.89
8,Cádiz,Soria,3.667,629.91
9,Cádiz,La Rioja,3.703,721.72


In [57]:
print("--- Pares de Provincias desde Valencia con MENOS interacción ---")
con.execute("""
SELECT 
    provincia_origen,
    provincia_destino,
    viajes_totales,
    distancia_media_km
FROM gold_flujos_provinciales
WHERE provincia_origen = 'Valencia'
ORDER BY viajes_totales ASC
LIMIT 20
""").df()

--- Pares de Provincias desde Valencia con MENOS interacción ---


,provincia_origen,provincia_destino,viajes_totales,distancia_media_km
0,Valencia,Lugo,50.830,694.04
1,Valencia,Santa Cruz de Tenerife,55.624,1924.14
2,Valencia,Pontevedra,61.394,737.28
3,Valencia,Las Palmas,65.099,1766.44
4,Valencia,La Coruña,70.160,768.57
5,Valencia,Cádiz,73.410,547.09
6,Valencia,Asturias,142.438,547.06
7,Valencia,Álava,189.696,433.64
8,Valencia,Vizcaya,189.919,489.39
9,Valencia,Guipúzcoa,307.452,442.85
